In [1]:
import json
import torch
import random
from PIL import Image
from tqdm import tqdm
import pickle

In [5]:
with open('/home/dhruv/Projects/VidSitu/vidsitu_data/vidsitu_annotations/vseg_ann_files/vsann_train_lb.json', 'r') as f:
    train = json.load(f)

In [4]:
with open('/home/dhruv/Projects/VidSitu/vidsitu_data/vidsitu_annotations/vseg_ann_files/vsann_valid_lb.json', 'r') as f:
    valid = json.load(f)

In [8]:
train[0]

{'Ev1': {'vid_uid': 'M3u94uEBq9o5',
  'vid_seg_int': 'v_M3u94uEBq9o_seg_80_90',
  'EvRel': 'Causes',
  'Args': {'Arg1 (thing falling)': 'man in wetsuit',
   'ArgM (direction)': 'downward',
   'Scene of the Event': 'lake'},
  'Arg_List': {'Arg1 (thing falling)': '0',
   'ArgM (direction)': '1',
   'Scene of the Event': '2'},
  'Verb': 'dive (to drop)',
  'VerbID': 'dive.02'},
 'Ev2': {'vid_uid': 'M3u94uEBq9o5',
  'vid_seg_int': 'v_M3u94uEBq9o_seg_80_90',
  'EvRel': 'Causes',
  'Args': {'Arg0 (breather)': 'man in wetsuit',
   'Arg1 (air, we hope)': 'man in wetsuit',
   'ArgM (manner)': 'aggressively',
   'Scene of the Event': 'lake'},
  'Arg_List': {'Arg0 (breather)': '0',
   'Arg1 (air, we hope)': '1',
   'ArgM (manner)': '2',
   'Scene of the Event': '3'},
  'Verb': 'breathe (suck air)',
  'VerbID': 'breathe.01'},
 'Ev3': {'vid_uid': 'M3u94uEBq9o5',
  'vid_seg_int': 'v_M3u94uEBq9o_seg_80_90',
  'Args': {'Arg0 (talker)': 'brunette girl',
   'Arg2 (hearer)': 'brunette boy',
   'ArgM (loc

In [28]:
def convert_to_fine_tuning_format(dataset):
    result = []

    for item in dataset:
        for event_key, event_data in item.items():
            #vid_uid = event_data['vid_uid']
            vid_seg_int = event_data['vid_seg_int']
            verb = event_data['Verb']
            roles = {i+1: role for i, role in enumerate(event_data['Args'].keys())}
            nouns = {role: noun for role, noun in event_data['Args'].items()}
            id_ = f"{vid_seg_int}_{event_key}"

            # Calculate start and end times for the event
            event_index = int(event_key[2:]) - 1  # Extract event index from key (Ev1 -> 0, Ev2 -> 1, etc.)
            start_time = event_index * 2
            end_time = start_time + 2

            result.append({
                'id': id_,
                'video': f"{vid_seg_int}.mp4",  # Update the path according to your image storage structure
                'event': event_key,
                'start_secs': start_time,
                'end_secs': end_time,
                'conversations': [
                    {
                        'from': 'human',
                        'value': "<video>\n Task VidSitu: Generate the verb and corresponding roles, then generate the noun based on the verb and roles"
                    },
                    {
                        'from': 'gpt',
                        'value': json.dumps({"verb": verb, "roles": roles, "nouns": nouns}, ensure_ascii=False)
                    }
                ]
            })
    
    return result

In [29]:
fine_tuning_dataset = convert_to_fine_tuning_format(train)
#print(json.dumps(fine_tuning_dataset, indent=2))

In [26]:
fine_tuning_dataset

[{'id': 'v_M3u94uEBq9o_seg_80_90_Ev1',
  'video': '/home/dhruv/Projects/VidSitu/vidsitu_data/vsitu_video_trimmed_dir/v_M3u94uEBq9o_seg_80_90.mp4',
  'event': 'Ev1',
  'start_secs': 0,
  'end_secs': 2,
  'conversations': [{'from': 'human',
    'value': '<video>\n Task VidSitu: Generate the verb and corresponding roles, then generate the noun based on the verb and roles'},
   {'from': 'gpt',
    'value': '{"verb": "dive (to drop)", "roles": {"1": "Arg1 (thing falling)", "2": "ArgM (direction)", "3": "Scene of the Event"}, "nouns": {"Arg1 (thing falling)": "man in wetsuit", "ArgM (direction)": "downward", "Scene of the Event": "lake"}}'}]},
 {'id': 'v_M3u94uEBq9o_seg_80_90_Ev2',
  'video': '/home/dhruv/Projects/VidSitu/vidsitu_data/vsitu_video_trimmed_dir/v_M3u94uEBq9o_seg_80_90.mp4',
  'event': 'Ev2',
  'start_secs': 2,
  'end_secs': 4,
  'conversations': [{'from': 'human',
    'value': '<video>\n Task VidSitu: Generate the verb and corresponding roles, then generate the noun based on th

In [13]:

# Convert dataset
fine_tuning_data = convert_to_fine_tuning_format(train_data)

In [12]:
len(fine_tuning_dataset)

121330

In [30]:
output_file = "vidsitu_train_SFT.json"
with open(output_file, "w") as f:
    json.dump(fine_tuning_dataset, f, indent=2)

In [7]:

output_file = "imsitu_train_SFT.json"

fine_tuning_data = json.load(open(output_file))

In [9]:
unique_images = {}

# Iterate through the dataset and randomly select one element per unique image
for item in fine_tuning_data:
    image_id = item['id']
    if image_id not in unique_images:
        unique_images[image_id] = []
    unique_images[image_id].append(item)

# Select one random element for each unique image
final_data = []
for image_id, items in unique_images.items():
    selected_item = random.choice(items)
    final_data.append(selected_item)

In [13]:
output_file = "imsitu_train_unique_SFT.json"
with open(output_file, "w") as f:
    json.dump(final_data, f, indent=2)

In [10]:
len(final_data)

75702

In [11]:
75702*3

227106

In [ ]:

# Print or save the modified dataset
print(json.dumps(final_data, indent=2))